In [1]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
import pdb
PATH = "C:\Program Files (x86)\chromedriver.exe"

In [2]:
def extract(item):
    atag = item.h2.a
    url = 'https://amazon.co.uk' + atag.get('href')
    
    driver.get(url)
    pagesoup = BeautifulSoup(driver.page_source,'html.parser')
    try:
        price = pagesoup.find('span',attrs= {'id':'price_inside_buybox'}).text.strip()
        table = pagesoup.find("table",attrs = {'class':'a-normal a-spacing-micro'})
        table = table.find_all("td")
    except AttributeError:
        return
    
    try:
        rating = pagesoup.find('span',attrs={'class':'a-icon-alt'}).text
        reviews = pagesoup.find('span',attrs={'class':"a-size-base",'id':'acrCustomerReviewText'}).text
    except AttributeError:
        rating  = 'NaN'
        reviews = 0
    
    
    properties = {'Price':price,'Reviews':reviews,'Rating':rating}
    for i in range(len(table)):
        if(i%2 == 1):
            prev  = attr
            attr  = table[i].text.strip()
            properties[prev] = attr
        else:
            attr  = table[i].text.strip()
    return properties
        
    

In [31]:
records = []
for i in range(1,21):
    driver = webdriver.Chrome(PATH)
    URL = 'https://www.amazon.co.uk/s?k=laptops&page='+str(i)+'&qid=1615761982&ref=sr_pg_'+str(i)
    driver.get(URL)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    results = soup.find_all('div',{'data-component-type':'s-search-result'})
    for item in results:
        record  = extract(item)
        if record:
            records.append(record)

In [40]:
attrs = []
for record in records:
    keys = record.keys()
    for key in keys:
        if key not in attrs:
            attrs.append(key)

In [41]:
attrs

['Price',
 'Reviews',
 'Rating',
 'Brand',
 'Operating System',
 'CPU manufacturer',
 'Screen size',
 'Computer memory size',
 'Processor count',
 'Graphics Chipset Brand',
 'Series',
 'CPU model',
 'Colour',
 'Hard disk size',
 'Resolution',
 'Item weight',
 'Form Factor',
 'Hard disk description',
 'Operating system',
 'Graphics processor manufacturer',
 'Model name',
 'RAM memory technology',
 'Hard disk interface',
 'CPU speed',
 'Form factor',
 'Wireless communication technology',
 'Human interface input',
 'Connectivity Type',
 'Graphics co-processor',
 'Battery cell composition',
 'Item dimensions L x W x H',
 'Cellular technology',
 'Other camera features',
 'Software included',
 'Wireless communication standard']

In [50]:
with open('results.csv','w',newline = '',encoding = 'utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(attrs)
    for record in records:
        row = []
        for attr in attrs:
            try:
                val = record[attr]
            except KeyError:
                val = 'NaN'
            row.append(val)
        writer.writerow(row)